<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

### NISAR DEM notebook & Papermill example
#### Kernel: isce

This notebook downloads the specified H5 file and generates elevation contour images. The cell annotated with the 'parameters' tag below indicates the input parameters for Papermill execution.

### Be sure [default] credentials in ~/.aws/credentials are valid - use aws-login to renew if necessary.

Note: An error about a missing .cache directory may come up. This is due to a bug in the Black library:
https://github.com/psf/black/issues/1223

#### Initial imports
Some logic is also performed to direct downloads and data generated by this notebook to the *notebook_output* subdirectory.

In [ ]:
import os
from osgeo import gdal
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

# this block makes sure the directory set-up/change is only done once and relative to the notebook's directory
try:
    start_dir
except NameError:
    start_dir = os.getcwd()
    !mkdir -p ./notebook_output/04-NISAR_DEM_notebook
    os.chdir('notebook_output/04-NISAR_DEM_notebook')

python_dir = os.path.join(start_dir, 'python')

### Papermill parameters needed to run this notebook

```
papermill \
    04-NISAR_DEM_notebook.ipynb \
    04-NISAR_DEM_notebook_output.ipynb \
    -p url 'https://cae-artifactory.jpl.nasa.gov/artifactory/general-develop/gov/nasa/jpl/nisar/adt/r1/Data_GSLC_GCOV/UAVSAR_SanAnd.tar.gz!/UAVSAR_SanAnd/input/SanAnd_05024_18038_006_180730_L090_CX_129_05.h5' \
    -k isce \
    --log-output
```

In [ ]:
# will generate a new "parameters" cell through papermill

# url to download .h5 
# ie. 'https://cae-artifactory.jpl.nasa.gov/artifactory/general-develop/gov/nasa/jpl/nisar/adt/r1/Data_GSLC_GCOV/UAVSAR_SanAnd.tar.gz!/UAVSAR_SanAnd/input/SanAnd_05024_18038_006_180730_L090_CX_129_05.h5'
url = 'https://cae-artifactory.jpl.nasa.gov/artifactory/general-develop/gov/nasa/jpl/nisar/adt/r1/Data_GSLC_GCOV/UAVSAR_SanAnd.tar.gz!/UAVSAR_SanAnd/input/SanAnd_05024_18038_006_180730_L090_CX_129_05.h5'

In [ ]:
# downloading h5 file from url
!wget $url

### Using DEM.py file to pull BBOX and generate .tiff file

In [ ]:
# getting the h5 file from the url, ie. SanAnd_05024_18038_006_180730_L090_CX_129_05.h5
h5_file = url.split('/')[-1]

!python $python_dir/DEM.py --product $h5_file

### Rest of the code is pulled from earthdatascience.org

https://www.earthdatascience.org/tutorials/visualize-digital-elevation-model-contours-matplotlib/

In [ ]:
filename = "dem.tif"
gdal_data = gdal.Open(filename)
gdal_band = gdal_data.GetRasterBand(1)
nodataval = gdal_band.GetNoDataValue()

In [ ]:
# convert to a numpy array
data_array = gdal_data.ReadAsArray().astype(np.float64)
data_array

In [ ]:
# replace missing values if necessary
if np.any(data_array == nodataval):
    data_array[data_array == nodataval] = np.nan

In [ ]:
#Plot out data with Matplotlib's 'contour'
fig = plt.figure(figsize = (12, 10))
# ax = fig.add_subplot(111)
plt.contour(data_array, 
            cmap = "viridis", 
            levels = list(range(0, 5000, 100))
           )
plt.title("Elevation Contours of NISAR sampe of San Andreas")
cbar = plt.colorbar()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
#Plot our data with Matplotlib's 'contourf'
fig = plt.figure(figsize = (12, 10))
ax = fig.add_subplot(111)
plt.contourf(data_array, 
             cmap = "viridis", 
             levels = list(range(0, 5000, 500))
            )
plt.title("Elevation Contours of NISAR sampe of San Andreas")
cbar = plt.colorbar()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.7.1 and above</font>